## 토큰화

### 파일 로드

In [ ]:
import pandas as pd
import os
import gc
from collections import Counter

base_dir = os.getcwd() + "/"

base_dir

#base_dir = "/content/drive/MyDrive/bookend/dev/text-style-classify/basic/"

In [ ]:
df1 = pd.read_csv(base_dir + "datasets/translate.csv")
"""df2 = pd.read_csv(base_dir + "datasets/written_0.csv")
df3 = pd.read_csv(base_dir + "datasets/written_1.csv")
df4 = pd.read_csv(base_dir + "datasets/written_2.csv")
df5 = pd.read_csv(base_dir + "datasets/written_3.csv")"""

### BERT 토크나이저

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

def bert_tokenizer(sentence):
    encoding = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=64,
        padding="max_length",
        return_attention_mask=False,
        return_tensors='pt',
        truncation=True
    )
    tokens = encoding['input_ids'][0]
    token_list = tokenizer.convert_ids_to_tokens(tokens)
    return pd.Series([tokens, token_list])

In [ ]:
#df1[["bert_encodes", "bert_tokens"]] = df1["sentence"].apply(bert_tokenizer)

In [ ]:
"""df2[["bert_encodes", "bert_tokens"]] = df2["sentence"].apply(bert_tokenizer)"""

### Mecab 토크나이저

In [ ]:
from mecab import MeCab

mecab = MeCab()

def mecab_tokenizer(sentence):
    tokens = mecab.morphs(sentence)
    return tokens

In [ ]:
"""df1["mecab_tokens"] = df1["sentence"].apply(mecab_tokenizer)
df1.to_pickle(base_dir + "datasets/translate.pkl")"""

### Tiktokenizer

In [ ]:
import tiktoken

# 사용할 인코딩 선택
encoding_name = "cl100k_base"  # "p50k_base", "r50k_base", "gpt2" 등으로 변경 가능

# tiktoken 토크나이저 초기화
tokenizer = tiktoken.get_encoding(encoding_name)

def tiktoken_tokenizer(sentence):
    # 문자열이 아닌 경우 빈 시리즈 반환
    if not isinstance(sentence, str):
        return pd.Series([[], []])
    
    # tiktoken을 사용하여 문장을 토큰화하고 ID로 변환
    token_ids = tokenizer.encode(sentence)
    # 토큰 ID를 다시 토큰으로 변환
    token_list = [tokenizer.decode([token_id])[0] for token_id in token_ids]
    return pd.Series([token_ids, token_list])

In [ ]:
df1 = pd.read_csv(base_dir + "datasets/translate.csv")
df1[["tiktoken_encodes", "tiktoken_tokens"]] = df1["sentence"].apply(tiktoken_tokenizer)
df1.to_pickle(base_dir + "datasets/translate.pkl")

### 토큰 인코딩

In [ ]:
vocab_counter = Counter()
print("start")

# 첫 번째 데이터셋 처리
df = pd.read_pickle(base_dir + "compact_datasets/translate.pkl")
tokenized_sentences = df['tiktoken_encodes']

for sentence in tokenized_sentences:
    for word in sentence:
        vocab_counter[word] += 1

del tokenized_sentences
gc.collect()

# 나머지 데이터셋 처리
for i in range(20):
    print(f"file : {i}")
    df = pd.read_pickle(base_dir + f"compact_datasets/written_{i}.pkl")
    tokenized_sentences = df['tiktoken_encodes']
    
    for sentence in tokenized_sentences:
        for word in sentence:
            if type(word) == int:
                vocab_counter[word] += 1
            else:
                print(f"error {word}")

    del tokenized_sentences
    gc.collect()


print(f"Total unique words: {len(vocab_counter)}")


In [ ]:
# 상위 3만 개 단어 추출
top_30000_words = vocab_counter.most_common(30000)

word_to_index = {word: index+2 for index, (word, _) in enumerate(top_30000_words)}
word_to_index['PAD'] = 0
word_to_index['OOV'] = 1

# dictionary 출력 확인
print(f"Top 30000 words mapping: {word_to_index}")

# dictionary를 파일로 저장 (옵션)
import pickle
with open(base_dir + "compact_datasets/vocab_ver1.pkl", "wb") as f:
    pickle.dump(word_to_index, f)

In [9]:
# 데이터프레임에 적용
df = pd.read_pickle(base_dir + "compact_datasets/translate.pkl")
df['encoded_sentence'] = df['tiktoken_encodes'].apply(lambda x: [word_to_index.get(word, 1) for word in x])
df.to_pickle(base_dir + "compact_datasets/translate.pkl")

del df
gc.collect()

for i in range(20):
    print(i)
    df = pd.read_pickle(base_dir + f"compact_datasets/written_{i}.pkl")
    df['encoded_sentence'] = df['tiktoken_encodes'].apply(lambda x: [word_to_index.get(word, 1) for word in x])
    df.to_pickle(base_dir + f"compact_datasets/written_{i}.pkl")
    del df
    gc.collect()

## author 인코딩

In [ ]:
df1 = pd.read_pickle(base_dir + "datasets/translate.pkl")
df2 = pd.read_pickle(base_dir + "datasets/written_0.pkl")

In [ ]:
author_set = set(df1['author'].to_list())

df1 = None
gc.collect()


for i in range(20):
    print(i)
    gc.collect()
    df = pd.read_pickle(base_dir + "datasets/written_" + str(i) + ".pkl")
    temp_set = set(df['author'].to_list())
    author_set = author_set.union(temp_set)
    df = None


author_dict = {author : i for i, author in enumerate(author_set)}

len(author_dict)

In [ ]:
import json

with open(base_dir + "datasets/author_encoding.json", "w") as json_file:
    json.dump(author_dict, json_file, indent=4)

df1 = pd.read_pickle(base_dir + "datasets/translate.pkl")
df1['encoded_author'] = df1['author'].apply(lambda x : author_dict[x])
df1.to_pickle(base_dir + "datasets/translate.pkl")

df1 = None
gc.collect()


for i in range(20):
    df = pd.read_pickle(base_dir + "datasets/written_" + str(i) + ".pkl")

    df['encoded_author'] = df['author'].apply(lambda x : author_dict[x])

    df.to_pickle(base_dir + "datasets/written_" + str(i) + ".pkl") 

    df = None

    gc.collect()


len(author_set)

In [ ]:
df = pd.read_pickle(base_dir + "datasets/translate.pkl")
df.to_csv(base_dir + "datasets/translate.csv", index=False, encoding='utf-8-sig')
df = None
gc.collect()

for i in range(20):
    print(i)
    df = pd.read_pickle(base_dir + "datasets/written_" + str(i) + ".pkl")
    df.to_csv(base_dir + "datasets/written_" + str(i) + ".csv", index=False, encoding='utf-8-sig')
    df = None
    gc.collect()

## Compactize

In [ ]:
df = pd.read_pickle(base_dir + "datasets/translate.pkl")
df = df[['tiktoken_encodes', 'encoded_author']]
df.to_pickle(base_dir + "compact_datasets/translate.pkl")

del df
gc.callbacks

In [ ]:
for i in range(20):
    print(i)
    df = pd.read_pickle(base_dir + "datasets/written_" + str(i) + ".pkl")
    df = df[['tiktoken_encodes', 'encoded_author']]
    df.to_pickle(base_dir + "compact_datasets/written_" + str(i) + ".pkl")
    df = None
    gc.collect()

### 장르 인코딩

In [ ]:
#df = pd.read_pickle(base_dir +  f'data/all.pkl')

In [ ]:
df['class'] = df['Classify_1']+"/" + df['Classify_2']

classify_set = set(df['class'].to_list())

classify_dict = {classs : i for i, classs in enumerate(classify_set)}

import json

with open(base_dir + "data/class_encoding.json", "w") as json_file:
    json.dump(classify_dict, json_file, indent=4)

df['encoded_class'] = df['class'].apply(lambda x : classify_dict[x])

In [ ]:
df.to_pickle(base_dir +  f'data/labeled_compact.pkl')